In [42]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils

In [43]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [55]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [56]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [17]:
class SampledSoftmax(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(SampledSoftmax, self).__init__(**kwargs)

    def build(self, input_shape):
        # assert isinstance(input_shape, list)
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[0][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, x):
        # assert isinstance(x, list)
        a, b = x
        return [K.dot(a, self.kernel) + b, K.mean(b, axis=-1)]

    def compute_output_shape(self, input_shape):
        # assert isinstance(input_shape, list)
        shape_a, shape_b = input_shape
        return [(shape_a[0], self.output_dim), shape_b[:-1]]

In [15]:
def my_very_own_softmax(prediction_tensor, target_tensor):
    print(prediction_tensor, target_tensor)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=target_tensor,
        logits=prediction_tensor)
    return tf.reduce_sum(cross_entropy)

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5);

Epoch 1/5
60000/60000 [==============================] - 6s 95us/step - loss: 0.7331 - acc: 0.7122
Epoch 2/5
60000/60000 [==============================] - 4s 74us/step - loss: 0.0790 - acc: 0.9816
Epoch 3/5
60000/60000 [==============================] - 4s 74us/step - loss: 0.0472 - acc: 0.9883
Epoch 4/5
60000/60000 [==============================] - 4s 74us/step - loss: 0.0337 - acc: 0.9908
Epoch 5/5
60000/60000 [==============================] - 4s 75us/step - loss: 0.0256 - acc: 0.9926


In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 0s 28us/step


In [11]:
print(test_acc)

0.9907


---

In [14]:
import tensorflow as tf
from tensorflow.python.ops import array_ops

def focal_loss(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.softmax(prediction_tensor)
    # sigmoid_p = prediction_tensor
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

In [39]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        y_pred = tf.nn.softmax(y_pred)
        
        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [59]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, logits):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits))
        ce = tf.multiply(y_true, -tf.log(softmax))
        return tf.reduce_mean(ce)
    return focal_loss_fixed

In [60]:
model.compile(optimizer='adam',
              loss=focal_loss(),
              metrics=['accuracy'])

model.fit(train_images, utils.to_categorical(train_labels, 10), epochs=5, 
          validation_data=(test_images, utils.to_categorical(test_labels, 10)));

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 79us/step - loss: 0.3708 - acc: 0.9611 - val_loss: 0.3544 - val_acc: 0.9896
Epoch 2/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.3536 - acc: 0.9879 - val_loss: 0.3515 - val_acc: 0.9911
Epoch 3/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.3510 - acc: 0.9917 - val_loss: 0.3504 - val_acc: 0.9925
Epoch 4/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.3498 - acc: 0.9935 - val_loss: 0.3496 - val_acc: 0.9926
Epoch 5/5
60000/60000 [==============================] - 4s 67us/step - loss: 0.3489 - acc: 0.9952 - val_loss: 0.3495 - val_acc: 0.9927


In [73]:
test_loss, test_acc = model.evaluate(test_images, utils.to_categorical(test_labels, 10))

10000/10000 [==============================] - 1s 78us/step


In [74]:
print(test_acc)

0.9927


In [69]:
model.add(tf.keras.layers.Activation('softmax'))

In [66]:
test_labels[3]

0

In [72]:
model.compile(optimizer='adam',
              loss=focal_loss(),
              metrics=['accuracy'])